In [ ]:
import sys
sys.path.append("..") 
from smartbind.preprocess import convert_smiles_to_pf2
from smartbind import BindingPL
import os

In [ ]:
pos_chain = 'GACAGCUGCUGUC'
A1AZL = 'c1cc(ccc1C2=NCCN2CCN)C(=O)Nc3ccc(cc3)C4=NCCN4'
A1AZL_fp2 = convert_smiles_to_pf2(A1AZL)

In [ ]:
def predict_binding(target_rna_chain, ligand_fp2, best_model_paths):
    binding_site_predictions_dict = {i: [] for i in range(len(target_rna_chain))}
    for weight_name in os.listdir(best_model_paths):
        weight_path = os.path.join(best_model_paths, weight_name)
        smartbind_binding_model = BindingPL(device='cpu', vs_mode=True)
        smartbind_binding_model.load_pretrained_model(model_path=weight_path, device='cpu', mode='inference')
        binding_prediction = smartbind_binding_model.predict_binding(target_rna_chain, ligand_fp2)
        binding_prediction = binding_prediction.detach().numpy()
        normalized_binding_prediction = (binding_prediction - min(binding_prediction)) / (max(binding_prediction) - min(binding_prediction)) 
        for i in range(len(target_rna_chain)):
            binding_site_predictions_dict[i].append(normalized_binding_prediction[i])

    average_predictions = []
    for i in range(len(target_rna_chain)):
        average_predictions.append(sum(binding_site_predictions_dict[i]) / 10)

    normalized_binding_site_prediction = ((average_predictions - min(average_predictions)) /
                                        (max(average_predictions) - min(average_predictions)))
    normalized_binding_site_prediction = [round(i, 3) for i in normalized_binding_site_prediction]
    normalized_binding_site_prediction = [float(i) for i in normalized_binding_site_prediction]
    return normalized_binding_site_prediction

In [ ]:
normalized_binding_site_prediction = predict_binding(pos_chain, A1AZL_fp2, '../SMARTBind_weight')
normalized_binding_site_prediction = [round(i, 3) for i in normalized_binding_site_prediction]
str(normalized_binding_site_prediction)